In [ ]:
import time
import pandas as pd
import fbprophet 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

# total population by place
pop_by_place=pd.read_csv('../../data/NHGIS/nhgis0002_csv/nhgis0002_ts_nominal_place.csv',encoding='ISO-8859-1')

In [13]:
pop_by_place.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31436 entries, 0 to 31435
Data columns (total 25 columns):
NHGISCODE    31436 non-null object
GJOIN1970    20449 non-null object
GJOIN1980    22529 non-null object
GJOIN1990    23435 non-null object
GJOIN2000    25150 non-null object
GJOIN2010    29514 non-null object
GJOIN2012    29509 non-null object
STATE        31436 non-null object
STATEFP      31436 non-null int64
STATENH      31436 non-null int64
PLACE        31436 non-null object
PLACEA       31436 non-null int64
NAME1970     20449 non-null object
NAME1980     22529 non-null object
NAME1990     23435 non-null object
NAME2000     25150 non-null object
NAME2010     29514 non-null object
NAME2012     29509 non-null object
AV0AA1970    6435 non-null float64
AV0AA1980    22529 non-null float64
AV0AA1990    23435 non-null float64
AV0AA2000    25150 non-null float64
AV0AA2010    29514 non-null float64
AV0AA125     29509 non-null float64
AV0AA125M    29509 non-null float64
dtypes: float

In [ ]:
nyc=pop_by_place.iloc[[18672]]
nyc

In [ ]:
pop_by_place_ = pop_by_place.copy()[['NHGISCODE','AV0AA1970','AV0AA1980','AV0AA2000','AV0AA2010']]
pop_by_place_#.sample(5)

- ***notes***:
    - forget places with only 1 measurement 

In [1]:
import pandas as pd
import fbprophet 

def population_by_place(years=20,n_places=1000,changepoint_prior=0.15,indicate=False):
    # total population by place (1970 to 2010)
    pop_by_place=pd.read_csv('../../data/NHGIS/nhgis0002_csv/nhgis0002_ts_nominal_place.csv',encoding='ISO-8859-1')
    """
    inputs) 
    >> years
        > number of years to forecast
    >> places
        > number of places to forecast +1 
            >> e.g. 99 = first 100 places (max==25102)
    >> changepoint_prior
        > set changepoint_prior_scale for prophet model
    >> indicate
        > default False
        > if True, print number of place forecasted after each forecast
    >> time
        > default False
        > if True, prints time the function took to run right before returning output
    
    function: 
    >> generate DataFrame of population:
        > from 1970 to 2010
        > by unique place (use NHGISCODE as Id)
    >> drop 
        > places with less than 2 measurements
            > can only predict places which have been measured 2+ times 
    >> extract list of places
        > each as a DataFrame ready for prediction 
        > column0='ds' , column1='y'
    >> make and fit prophet model on each place
    >> return prophet model's predictions
        > of each place
        > for {years} years
    """

    # df by NHGISCODE with measurements by decade (31436 rows × 5 columns)
    unique_places = pop_by_place.copy()[['NHGISCODE','AV0AA1970','AV0AA1980','AV0AA2000','AV0AA2010']]

    # drop NaN rows @ thresh = 3 due to NHGISCODE being non-NaN (25103 rows × 5 columns ; 6333 non-measurable) 
    measureable_unique_places = unique_places.dropna(axis=0,thresh=3)
    # convert NaN values to 0 (note: there are 270 'dead' counties ('A00AA2010' == 0))
    measureable_unique_places = measureable_unique_places.fillna(0)

    # generate list of remaining NHGISCODE codes 
    codes_of_measureable_unique_places = [code for code in measureable_unique_places.NHGISCODE]
    # drop NHGISCODE column (25103 rows × 4 columns)
    measureable_unique_places = measureable_unique_places.drop('NHGISCODE',axis=1)

    # list of str column names as years (for conversion to datetime)
    year_only_columns = [i[5:] for i in measureable_unique_places.columns]
    # convert year_only_columns to DatetimeIndex of Timestamps
    dt_columns = pd.to_datetime(arg=year_only_columns)

    # convert dt_columns into dataframe 
    datetime_df = pd.DataFrame(dt_columns).T
    # w/ columns, so concatable with measureable_unique_counties
    datetime_df.columns = measureable_unique_places.columns

    # generate list of remaining places (each as pd.Series)
    dfs_of_measureable_unique_places = [measureable_unique_places.iloc[place] for place in range(len(measureable_unique_places))]

    # add datetime_df to each dataframe as first row
    prophet_places = [pd.concat((datetime_df,pd.DataFrame(place).T),axis=0) for place in dfs_of_measureable_unique_places]
    # then transpose to 2 rows x 23 columns 
    prophet_almost_ready_places = [place.T for place in prophet_places]

    # set collection of prophets 
    prophet_by_place = []

    # run prophet model on first 1000 places
    for place in range(len(prophet_almost_ready_places[:n_places])):
        # make the prophet model
        place_prophet = fbprophet.Prophet(changepoint_prior_scale=changepoint_prior)
        # identify county 
        a = prophet_almost_ready_places[place]
        # rename place df's columns to agree with prophet formatting
        a.columns = ['ds','y']
        # fit place on prophet model 
        b = place_prophet.fit(a)
        # make a future dataframe for 20 years
        place_forecast = place_prophet.make_future_dataframe( periods=1*years, freq='Y' )
        # establish predictions
        place_forecast = place_prophet.predict(place_forecast)
        # add to collection 
        prophet_by_place.append(place_forecast)
        # did we ask for indication (hint: do this if calculating for > 1000 places unless you enjoy anxiety)
        if indicate==True:
            # let us know the count
            print(place)
        
    # return forecasts
    return prophet_by_place

In [2]:
by_place = population_by_place(indicate=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 2.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 2.0.


0


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 2.0.


1


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 2.0.


...

996


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 2.0.


997


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 2.0.


998
999


In [3]:
len(by_place)

1000

In [54]:
by_place[:10]

[           ds        trend   yhat_lower   yhat_upper  trend_lower  \
 0  1970-01-01  2487.150876  2996.000000  2996.000000  2487.150876   
 1  1980-01-01  2657.875167  3155.000000  3155.000000  2657.875167   
 2  2000-01-01  2489.875167  2987.000000  2987.000000  2489.875167   
 3  2010-01-01  2179.150876  2688.000000  2688.000000  2179.150876   
 4  2010-12-31  2148.189025  2625.116839  2625.116839  2148.189025   
 5  2011-12-31  2117.142114  2586.673761  2586.673761  2117.142114   
 6  2012-12-31  2086.010143  2576.742940  2576.742940  2086.010143   
 7  2013-12-31  2054.963232  2538.960959  2538.960959  2054.963232   
 8  2014-12-31  2023.916321  2500.825895  2506.730335  2023.898081   
 9  2015-12-31  1992.869409  2451.336745  2481.226198  1981.805098   
 10 2016-12-31  1961.737438  2431.269058  2482.445530  1940.536260   
 11 2017-12-31  1930.690527  2379.300516  2458.686604  1895.302788   
 12 2018-12-31  1899.643616  2327.133553  2440.061452  1850.205739   
 13 2019-12-31  1868

In [59]:
# df by NHGISCODE with measurements by decade (31436 rows × 5 columns)
unique_places = pop_by_place.copy()[['NHGISCODE','AV0AA1970','AV0AA1980','AV0AA2000','AV0AA2010']]

# drop NaN rows @ thresh = 3 due to NHGISCODE being non-NaN (25103 rows × 5 columns ; 6333 non-measurable) 
measureable_unique_places = unique_places.dropna(axis=0,thresh=3)
# convert NaN values to 0 (note: there are 270 'dead' counties ('A00AA2010' == 0))
measureable_unique_places = measureable_unique_places.fillna(0)

sample=[]
for code in measureable_unique_places[:10].NHGISCODE:
    a=pop_by_place.loc[pop_by_place['NHGISCODE'] == code]
    sample.append(a)

In [63]:
for i in range(len(sample)):
    o=sample[i][['PLACE','STATE']]
    print(o)

            PLACE    STATE
1  Abbeville city  Alabama
             PLACE    STATE
2  Adamsville city  Alabama
          PLACE    STATE
3  Addison town  Alabama
        PLACE    STATE
4  Akron town  Alabama
            PLACE    STATE
5  Alabaster city  Alabama
              PLACE    STATE
6  Albertville city  Alabama
                 PLACE    STATE
7  Alexander City city  Alabama
            PLACE    STATE
8  Alexandria CDP  Alabama
             PLACE    STATE
9  Aliceville city  Alabama
           PLACE    STATE
10  Allgood town  Alabama


In [74]:
"""
Abbeville city  Alabama; 
2,721 2586.673761
2,699 2576.742940
2,684 2538.960959
2,654 2500.844134
2,646 2462.401057
2,627 2452.470236
2,594 2414.688255
"""
print(by_place[0][['yhat']][5:12])

           yhat
5   2586.673761
6   2576.742940
7   2538.960959
8   2500.844134
9   2462.401057
10  2452.470236
11  2414.688255


In [28]:
pop_by_place=pd.read_csv('../../data/NHGIS/nhgis0002_csv/nhgis0002_ts_nominal_place.csv',encoding='ISO-8859-1')

unique_places = pop_by_place.copy()[['NHGISCODE','AV0AA1970','AV0AA1980','AV0AA2000','AV0AA2010']]

# drop NaN rows @ thresh = 3 due to NHGISCODE being non-NaN (25103 rows × 5 columns ; 6333 non-measurable) 
_measureable_unique_places = unique_places.dropna(axis=0,thresh=3)
# convert NaN values to 0 (note: there are 270 'dead' counties ('A00AA2010' == 0))
_measureable_unique_places = _measureable_unique_places.fillna(0)
test=_measureable_unique_places.NHGISCODE[:1000]
places_in_test=[]
for t in range(len(pop_by_place.NHGISCODE)):
    q = pop_by_place.copy()[['PLACE','STATE']].iloc[t]
    if pop_by_place.NHGISCODE[t] in test:
        places_in_test.append(q)    

In [ ]:
places_in_test

In [16]:
pop_by_place.NHGISCODE[1]

'G01000124'

In [18]:
pop_by_place.copy()[['PLACE','STATE']].iloc[1]

PLACE    Abbeville city
STATE           Alabama
Name: 1, dtype: object

In [27]:
# 'Abbeville city'

In [48]:
pop_by_place=pd.read_csv('../../data/NHGIS/nhgis0002_csv/nhgis0002_ts_nominal_place.csv',encoding='ISO-8859-1')
all_places = [place for place in pop_by_place.PLACE]
 

def forecast_this(place,n_years=20):
    a=0
    for p in all_places:
        if p == place:
            a=pop_by_place.loc[pop_by_place['PLACE'] == place]
        elif p == place + ' city':
            a=pop_by_place.loc[pop_by_place['PLACE'] == place + ' city']
    print(a.columns)
#         # make the prophet model
#         place_prophet = fbprophet.Prophet(changepoint_prior_scale=changepoint_prior)
#         # identify county 
#         a = prophet_almost_ready_places[place]
#         # rename place df's columns to agree with prophet formatting
#         a.columns = ['ds','y']
#         # fit place on prophet model 
#         b = place_prophet.fit(a)
#         # make a future dataframe for 20 years
#         place_forecast = place_prophet.make_future_dataframe( periods=1*n_years, freq='Y' )
#         # establish predictions
#         place_forecast = place_prophet.predict(place_forecast)
#         # add to collection 
#         prophet_by_place.append(place_forecast)
forecast_this('San Francisco')

Index(['NHGISCODE', 'GJOIN1970', 'GJOIN1980', 'GJOIN1990', 'GJOIN2000',
       'GJOIN2010', 'GJOIN2012', 'STATE', 'STATEFP', 'STATENH', 'PLACE',
       'PLACEA', 'NAME1970', 'NAME1980', 'NAME1990', 'NAME2000', 'NAME2010',
       'NAME2012', 'AV0AA1970', 'AV0AA1980', 'AV0AA1990', 'AV0AA2000',
       'AV0AA2010', 'AV0AA125', 'AV0AA125M'],
      dtype='object')


In [36]:
for i in all_places:
    if i == 'Abbeville city':
        print('ok')

ok
ok
ok
ok


In [85]:
actual7 = pd.read_csv('../../data/American_Community_Survey/ACS_17_5YR_S0101/ACS_17_5YR_S0101_with_ann.csv',encoding='ISO-8859-1',low_memory=False) 
actual6 = pd.read_csv('../../data/American_Community_Survey/ACS_16_5YR_S0101/ACS_16_5YR_S0101_with_ann.csv',encoding='ISO-8859-1',low_memory=False) 
actual5 = pd.read_csv('../../data/American_Community_Survey/ACS_15_5YR_S0101/ACS_15_5YR_S0101_with_ann.csv',encoding='ISO-8859-1',low_memory=False) 
actual4 = pd.read_csv('../../data/American_Community_Survey/ACS_14_5YR_S0101/ACS_14_5YR_S0101_with_ann.csv',encoding='ISO-8859-1',low_memory=False) 
actual3 = pd.read_csv('../../data/American_Community_Survey/ACS_13_5YR_S0101/ACS_13_5YR_S0101_with_ann.csv',encoding='ISO-8859-1',low_memory=False) 
actual2 = pd.read_csv('../../data/American_Community_Survey/ACS_12_5YR_S0101/ACS_12_5YR_S0101_with_ann.csv',encoding='ISO-8859-1',low_memory=False) 
actual1 = pd.read_csv('../../data/American_Community_Survey/ACS_11_5YR_S0101/ACS_11_5YR_S0101_with_ann.csv',encoding='ISO-8859-1',low_memory=False) 

In [87]:
a17 = actual7.copy()[['GEO.display-label','HC01_EST_VC01']][1:]
a16 = actual6.copy()[['GEO.display-label','HC01_EST_VC01']][1:]
a15 = actual5.copy()[['GEO.display-label','HC01_EST_VC01']][1:]
a14 = actual4.copy()[['GEO.display-label','HC01_EST_VC01']][1:]
a13 = actual3.copy()[['GEO.display-label','HC01_EST_VC01']][1:]
a12 = actual2.copy()[['GEO.display-label','HC01_EST_VC01']][1:]
a11 = actual1.copy()[['GEO.display-label','HC01_EST_VC01']][1:]

len(a17),len(a16),len(a15),len(a14),len(a13),len(a12),len(a11)

(29576, 29574, 29574, 29549, 29509, 29509, 29517)

In [93]:
a17places = [place for place in a17['GEO.display-label']]
a16places = [place for place in a16['GEO.display-label']]
a15places = [place for place in a15['GEO.display-label']]
a14places = [place for place in a14['GEO.display-label']]
a13places = [place for place in a13['GEO.display-label']]
a12places = [place for place in a12['GEO.display-label']]
a11places = [place for place in a11['GEO.display-label']]

In [95]:
combo_places=[]
for place in a17places:
    if place in a16places:
        if place in a15places:
            if place in a14places:
                if place in a13places:
                    if place in a12places:
                        if place in a11places:
                            combo_places.append(place)

In [96]:
len(combo_places)

29341

In [97]:
combo_places

['Abanda CDP, Alabama',
 'Abbeville city, Alabama',
 'Adamsville city, Alabama',
 'Addison town, Alabama',
 'Akron town, Alabama',
 'Alabaster city, Alabama',
 'Albertville city, Alabama',
 'Alexander City city, Alabama',
 'Alexandria CDP, Alabama',
 'Aliceville city, Alabama',
 'Allgood town, Alabama',
 'Altoona town, Alabama',
 'Andalusia city, Alabama',
 'Anderson town, Alabama',
 'Anniston city, Alabama',
 'Arab city, Alabama',
 'Ardmore town, Alabama',
 'Argo town, Alabama',
 'Ariton town, Alabama',
 'Arley town, Alabama',
 'Ashford town, Alabama',
 'Ashland town, Alabama',
 'Ashville city, Alabama',
 'Athens city, Alabama',
 'Atmore city, Alabama',
 'Attalla city, Alabama',
 'Auburn city, Alabama',
 'Autaugaville town, Alabama',
 'Avon town, Alabama',
 'Axis CDP, Alabama',
 'Babbie town, Alabama',
 'Baileyton town, Alabama',
 'Bakerhill town, Alabama',
 'Ballplay CDP, Alabama',
 'Banks town, Alabama',
 'Bay Minette city, Alabama',
 'Bayou La Batre city, Alabama',
 'Bear Creek tow